In [31]:
import pandas as pd
import numpy as np
import os

In [88]:
sample = pd.read_csv('tune/' + 'veh' + '_dm_mrdr_os_shrinkage.csv')

col_sample = sample.columns[2:]

column = ['model']
for c in col_sample:
    column.append('[' + c.split('[')[1])

In [66]:
def get_smallest_mrdr(data):
    filename = 'tune/' + data + '_dm_mrdr_os_shrinkage.csv'
    data = pd.read_csv(filename)
    # switch
    switch_para = []
    for i in range(len(list(data['model']))):
        c = list(data['model'])[i]
        if 'switch' in c:
            switch_para.append(i)
    #shrinkage
    shrinkage_para = []
    for i in range(len(list(data['model']))):
        c = list(data['model'])[i]
        if 'shrinkage' in c:
            shrinkage_para.append(i)   
    #mrdr
    mrdr_para = []
    for i in range(len(list(data['model']))):
        c = list(data['model'])[i]
        if 'mrdr' in c:
            mrdr_para.append(i) 
    def get_smallest(dataframe, col_list):
        res = []
        for c in col_list:
            c_val = np.array(data.iloc[c][2:])
            res.append(c_val)
        return np.min(res,axis = 0)
    shrinkage_res = get_smallest(data, shrinkage_para)
    switch_res = get_smallest(data, switch_para)
    mrdr_res = get_smallest(data, mrdr_para)
    
    return mrdr_res, switch_res, shrinkage_res

In [67]:
def get_smallest_for_one_dataset(dataset):
    all_file = os.listdir('tune/')
    myfile = []
    for f in all_file:
        if 'robust_shrinkage.csv' in f and dataset in f and 'xshift' not in f:
            myfile.append(f)
    data = []
    for f in myfile:
        d = pd.read_csv('tune/' + f)
        data.append(d)
    model_name = data[0]['model']
    col_name = data[0].columns
    
    nparray = []
    for d in data:
        nparray.append(np.array(d[col_name[2:]]))
    one_df = np.concatenate(nparray,axis = 0)
    
    # switch + dmrobust
    col_index = []
    for i in range(len(model_name)):
        if 'switch' in model_name[i] and 'dmdr' in model_name[i]:
            col_index.append(i)
    switch_dmdr = np.min(one_df[col_index],axis = 0)
    
    # switch + robust
    col_index = []
    for i in range(len(model_name)):
        if 'switch' in model_name[i] and 'triple' in model_name[i]:
            col_index.append(i)
    switch_triple = np.min(one_df[col_index],axis = 0)
    
    
    # shrinkage + dmrobust
    col_index = []
    for i in range(len(model_name)):
        if 'shrinkage' in model_name[i] and 'dmdr' in model_name[i]:
            col_index.append(i)
    shrinkage_dmdr = np.min(one_df[col_index],axis = 0)
    
    # shrinkage + robust
    col_index = []
    for i in range(len(model_name)):
        if 'shrinkage' in model_name[i] and 'triple' in model_name[i]:
            col_index.append(i)
    shrinkage_triple = np.min(one_df[col_index],axis = 0)
    
    return switch_dmdr,switch_triple, shrinkage_dmdr, shrinkage_triple
        
                
    
    
    
    
    
    
    
    

In [112]:
def get_drpe(dataset):
    all_file = os.listdir('tune/')
    myfile = []
    for f in all_file:

        if 'drpe' in f and dataset in f and 'xshift' not in f:
            myfile.append(f)
    data = []
    for f in myfile:
        data.append(pd.read_csv('tune/' + f))
        
    col = data[0].columns[2:]
    min_res=[]
    for i in range(len(data)):
        min_res.append(np.array(data[i][col]))
    return np.min(min_res,axis = 0)


In [114]:
get_drpe('veh')[0]

array([0.11535, 0.05729, 0.09093, 0.05215, 0.29787, 0.28128, 0.1635 ,
       0.09766, 0.1204 , 0.06233, 0.23564, 0.14988])

In [91]:
def get_original_result(dataset,column):
    robustfile = 'cdf_result/'+dataset+'.csv'
    dmfile = 'tune/'+dataset+'_onehone_plotcdfv2.csv'
    
    robust = pd.read_csv(robustfile)
    dm = pd.read_csv(dmfile)[2:]
    dm.columns = robust.columns

    
    
    
    return  pd.concat([robust,dm],axis = 0)[dm.columns[1:]][column]
    
    
    

In [129]:
def get_final_file(dataname):
    original_file = get_original_result(dataname,column)

    switch_dmdr,switch_triple, shrinkage_dmdr, shrinkage_triple = get_smallest_for_one_dataset(dataname)

    mrdr, switch_dm, shrinkage_dm = get_smallest_mrdr(dataname)

    sinian = get_drpe(dataname)[0]







    original_file.loc[len(original_file)]=['mrdr'] + list(mrdr)
    original_file.loc[len(original_file)]=['switch dm'] + list(switch_dm)
    original_file.loc[len(original_file)]=['shrinkage dm'] + list(shrinkage_dm)

    original_file.loc[len(original_file)]=['switch dmdr'] + list(switch_dmdr)
    original_file.loc[len(original_file)]=['switch triple'] + list(switch_triple)
    original_file.loc[len(original_file)]=['shrinkage dmdr'] + list(shrinkage_dmdr)
    original_file.loc[len(original_file)]=['shrinkage triple'] + list(shrinkage_triple)
    original_file.loc[len(original_file)]=['sinian'] + list(sinian)
    
    original_file.to_csv('rebuttal_baseline/ps_' + dataname + '.csv')
    return original_file


In [130]:
print(get_final_file('opt'))

               model  [0.95, 0.1]_1  [0.95, 0.1]_0.9  [0.5, 0.1]_1  \
0          known ips        0.15407          0.05750       0.03320   
1        known snips        0.03971          0.01560       0.01402   
2     known dmrobust        0.01804          0.02263       0.37248   
3         known dmdr        0.03794          0.01481       0.01583   
4       known sndmdr        0.03555          0.01428       0.01170   
5       known robust        0.03162          0.02600       0.32758   
6       known triple        0.02802          0.02016       0.01487   
7     known sntriple        0.02637          0.02041       0.01154   
2           known dm        0.00897          0.01748       0.40905   
3         known dmdr        0.10344          0.03997       0.03021   
4         known sndr        0.08316          0.02817       0.01637   
11              mrdr        0.10637          0.19600       0.01036   
12         switch dm        0.02475          0.00471       0.39603   
13      shrinkage dm